In [1]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_community.chat_models import ChatOllama

from dotenv import load_dotenv

load_dotenv()


True

In [2]:
login = 'neo4j'
# password = 'XLR48xNsaLxKwVq'
password = 'your_password'

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = login
os.environ["NEO4J_PASSWORD"] = password


graph = Neo4jGraph()

In [3]:
model_path = r"C:\Work\Gazprom\LLM\llmChat\data\weights\starling-lm-7b-alpha.Q5_K_M.gguf"

rag_template: str = (
            'Отвечай на поставленные вопросы.\n'
            '\n'
            '{question}'
        )

rag_prompt = ChatPromptTemplate.from_template(rag_template)

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


In [ ]:

# model = LlamaCpp(
#     model_path=model_path, 
#     temperature=0.0,
#     max_new_tokens=10000,
#     context_window=16379-1000,
#     generate_kwargs={},
#     n_ctx=8192,
#     n_gpu_layers=-1, 
#     # n_threads=6, 
#     n_batch=512, 
#     verbose=True,
#     callback_manager=callback_manager
# )

model = ChatOllama(
    model="llama3.1",
    callback_manager = callback_manager, 
    temperature=0.0, 
    # num_predict = 128
)

In [17]:
loader = TextLoader(file_path=r"C:\Work\Gazprom\LLM\neo4j\dummytext.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
documents = text_splitter.split_documents(documents=docs)

In [18]:
llm_transformer = LLMGraphTransformer(llm=model)

In [19]:
"""
MATCH (n)
DETACH DELETE n
"""

import tqdm

def custom_convert_to_graph_documents(llm, documents, config = None):

    output_list = []

    N = len(documents)

    for i in tqdm.tqdm(range(N)):
        document = documents[i]
        # print(document)
        try:
            current_response = llm.process_response(document, config)
            output_list.append(current_response)
        except Exception as e:
            print(f'Error in doc: {document}\nError log: {e}')


    return output_list

# graph_documents = llm_transformer.convert_to_graph_documents(documents)
graph_documents = custom_convert_to_graph_documents(llm_transformer, documents)

  0%|          | 0/17 [00:00<?, ?it/s]

[{"text": "The Story of AmicoвЂ™s Family: A Legacy of Love and Tradition", "head": "Caruso family", "head_type": "Family", "relation": "IS_PART_OF", "tail": "Sicily", "tail_type": "Region"}, {"text": "The Carusos were not mere inhabitants of the land; they were the keepers of a culinary heritage that spanned generations.", "head": "Caruso family", "head_type": "Family", "relation": "HAS_HERITAGE", "tail": "culinary heritage", "tail_type": "Heritage"}, {"text": "Each family member contributed their unique flair, crafting a narrative of flavors that reflected their diverse experiences and deep-seated love for food.", "head": "Caruso family", "head_type": "Family", "relation": "HAS_MEMBER", "tail": "family members", "tail_type": "Group"}, {"text": "Giovanni Caruso and Maria: The Founding Generation", "head": "Giovanni Caruso", "head_type": "Person", "relation": "IS_PART_OF", "tail": "Caruso family", "tail_type": "Family"}, {"text": "Giovanni Caruso and Maria: The Founding Generation", "he

  6%|▌         | 1/17 [00:08<02:11,  8.21s/it]

[{"text": "Giovanni Caruso", "head": "Giovanni Caruso", "head_type": "Person", "relation": "PARENT_OF", "tail": "Maria", "tail_type": "Person"}, {"text": "Giovanni Caruso", "head": "Giovanni Caruso", "head_type": "Person", "relation": "FARMER_IN", "tail": "Santa Caterina", "tail_type": "Location"}, {"text": "Giovanni Caruso", "head": "Giovanni Caruso", "head_type": "Person", "relation": "PRODUCED", "tail": "olives and grapes", "tail_type": "Product"}, {"text": "Maria", "head": "Maria", "head_type": "Person", "relation": "WIFE_OF", "tail": "Giovanni Caruso", "tail_type": "Person"}, {"text": "Maria", "head": "Maria", "head_type": "Person", "relation": "COOK_IN", "tail": "the kitchen", "tail_type": "Location"}, {"text": "Antonio Caruso", "head": "Antonio Caruso", "head_type": "Person", "relation": "STORYTELLER_OF", "tail": "Giovanni Caruso and Maria", "tail_type": "Person"}]

 12%|█▏        | 2/17 [00:15<01:59,  7.93s/it]

...

 88%|████████▊ | 15/17 [02:11<00:17,  8.57s/it]

[{"head": "Caruso family", "head_type": "Family", "relation": "SUPPORTS", "tail": "Local Artists", "tail_type": "Group"}, {"head": "Caruso family", "head_type": "Family", "relation": "SUPPORTS", "tail": "Artisans", "tail_type": "Group"}, {"head": "Amico", "head_type": "Person", "relation": "ADVOCATES_FOR", "tail": "Local Artists", "tail_type": "Group"}, {"head": "Caruso family", "head_type": "Family", "relation": "COLLABORATES_WITH", "tail": "Local Musicians", "tail_type": "Group"}, {"head": "Caruso family", "head_type": "Family", "relation": "COLLABORATES_WITH", "tail": "Performers", "tail_type": "Group"}]

 94%|█████████▍| 16/17 [02:17<00:07,  7.68s/it]

[{"head": "Caruso family", "head_type": "Family", "relation": "HAS_MEMBER", "tail": "Each family member", "tail_type": "Person"},{"head": "Santa Caterina", "head_type": "Village", "relation": "LOCATED_IN", "tail": "New York City", "tail_type": "City"},{"head": "Caruso family", "head_type": "Family", "relation": "HAS_MEMBER", "tail": "Each family member", "tail_type": "Person"},{"head": "Santa Caterina", "head_type": "Village", "relation": "LOCATED_IN", "tail": "New York City", "tail_type": "City"},{"head": "Caruso family", "head_type": "Family", "relation": "HAS_LEGACY", "tail": "Culinary empire", "tail_type": "Business"},{"head": "Each family member", "head_type": "Person", "relation": "CONTRIBUTES_TO", "tail": "Caruso family legacy", "tail_type": "Legacy"},{"head": "Caruso family", "head_type": "Family", "relation": "IS_INVOLVED_IN", "tail": "Social causes", "tail_type": "Cause"}]

100%|██████████| 17/17 [02:24<00:00,  8.49s/it]


In [8]:
# from langchain_core.documents import Document

# text = """
# Marie Curie, born in 1867, was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity.
# She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.
# Her husband, Pierre Curie, was a co-winner of her first Nobel Prize, making them the first-ever married couple to win the Nobel Prize and launching the Curie family legacy of five Nobel Prizes.
# She was, in 1906, the first woman to become a professor at the University of Paris.
# """
# documents = [Document(page_content=text)]
# graph_documents = llm_transformer.convert_to_graph_documents(documents)

Llama.generate: 963 prefix-match hit, remaining 153 prompt tokens to eval

llama_print_timings:        load time =     213.14 ms
llama_print_timings:      sample time =      48.74 ms /   256 runs   (    0.19 ms per token,  5252.47 tokens per second)
llama_print_timings: prompt eval time =    1327.76 ms /   152 tokens (    8.74 ms per token,   114.48 tokens per second)
llama_print_timings:        eval time =    5560.54 ms /   256 runs   (   21.72 ms per token,    46.04 tokens per second)
llama_print_timings:       total time =    7091.54 ms /   408 tokens


KeyError: 'tail_type'

In [20]:
for ind, gd in enumerate(graph_documents):
    print(ind)
    try:
        gd.source.metadata['id']
    except:
        print('error')
        print(gd)

0
error
nodes=[Node(id='Caruso family', type='Family'), Node(id='culinary heritage', type='Heritage'), Node(id='Maria', type='Person'), Node(id='Giovanni Caruso', type='Person'), Node(id='Sicily', type='Region'), Node(id='family members', type='Group')] relationships=[Relationship(source=Node(id='Caruso family', type='Family'), target=Node(id='Sicily', type='Region'), type='IS_PART_OF'), Relationship(source=Node(id='Caruso family', type='Family'), target=Node(id='culinary heritage', type='Heritage'), type='HAS_HERITAGE'), Relationship(source=Node(id='Caruso family', type='Family'), target=Node(id='family members', type='Group'), type='HAS_MEMBER'), Relationship(source=Node(id='Giovanni Caruso', type='Person'), target=Node(id='Caruso family', type='Family'), type='IS_PART_OF'), Relationship(source=Node(id='Maria', type='Person'), target=Node(id='Caruso family', type='Family'), type='IS_PART_OF')] source=Document(metadata={'source': 'C:\\Work\\Gazprom\\LLM\\neo4j\\dummytext.txt'}, page_c

In [34]:
# graph.add_graph_documents(
#     graph_documents,
#     baseEntityLabel=True,
#     include_source=True
# )

for gd in graph_documents:
    try:
        graph.add_graph_documents(
            [gd],
            baseEntityLabel=True,
            include_source=True
        )
    except Exception as e:
        print(e)

{code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `apoc.create.addLabels`: Caused by: org.neo4j.internal.kernel.api.exceptions.schema.IllegalTokenNameException: '' is not a valid token name. Token names cannot be empty or contain any null-bytes.}


In [35]:
def showGraph():
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t").graph())
    widget.node_label_mapping = 'id'
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [30]:
from docx_splitter import split_doc_from_headers

documents1, _ = split_doc_from_headers(r'C:\Work\Gazprom\LLM\llmChat\data\original_docx\cand.docx')
documents2, _ = split_doc_from_headers(r'C:\Work\Gazprom\LLM\llmChat\data\original_docx\avr.docx')
documents3, _ = split_doc_from_headers(r'C:\Work\Gazprom\LLM\llmChat\data\original_docx\event.docx')

documents = documents1 + documents2 + documents3

In [31]:
graph_documents = custom_convert_to_graph_documents(llm_transformer, documents)

  0%|          | 0/61 [00:00<?, ?it/s]

[{"head": "кандидаты", "head_type": "Person/Company", "relation": "RELATED_TO", "tail": "форма", "tail_type": "Form"}, {"head": "форма", "head_type": "Form", "relation": "NEEDED_FOR", "tail": "отображение", "tail_type": "Action"}]

  2%|▏         | 1/61 [00:07<07:00,  7.02s/it]

[{"head": "кандидаты", "head_type": "Person", "relation": "HAS_SKILLS", "tail": "службы", "tail_type": "Service"},{"head": "службы", "head_type": "Service", "relation": "IS_RELATED_TO", "tail": "Бурение", "tail_type": "Drilling Service"},{"head": "службы", "head_type": "Service", "relation": "IS_RELATED_TO", "tail": "УДНГ", "tail_type": "Well Drilling Service"},{"head": "службы", "head_type": "Service", "relation": "IS_RELATED_TO", "tail": "Геология", "tail_type": "Geology Service"},{"head": "кандидаты", "head_type": "Person", "relation": "HAS_ACCESS_TO", "tail": "скважины", "tail_type": "Wells"},{"head": "скважины", "head_type": "Wells", "relation": "HAVE_CONDITIONS", "tail": "нефтяные/газовые/нагнетательные", "tail_type": "Well Conditions"}]

  3%|▎         | 2/61 [00:13<06:41,  6.81s/it]

...

 97%|█████████▋| 59/61 [08:17<00:17,  8.66s/it]

[{"head": "Excel", "head_type": "Software", "relation": "RELATED_TO", "tail": "Report", "tail_type": "Document"},{"head": "Excel", "head_type": "Software", "relation": "USED_FOR", "tail": "Creating Report", "tail_type": "Task"},{"head": "Button", "head_type": "UI Element", "relation": "PART_OF", "tail": "Excel Interface", "tail_type": "Interface"},{"head": "Flag", "head_type": "UI Element", "relation": "USED_FOR", "tail": "Confirming Report Generation", "tail_type": "Task"}]

 98%|█████████▊| 60/61 [08:22<00:07,  7.51s/it]

[{"head": "выполнение плана", "head_type": "concept", "relation": "DISPLAYED_AS", "tail": "факт/план", "tail_type": "format"}, {"head": "выполнение плана", "head_type": "concept", "relation": "SHOWS_DETAILS", "tail": "расчетные данные", "tail_type": "data"}, {"head": "выполнение плана", "head_type": "concept", "relation": "DISPLAYED_IN", "tail": "синяя диаграмма", "tail_type": "graph"}, {"head": "выполнение плана", "head_type": "concept", "relation": "HAS_STATISTICS", "tail": "таблицы", "tail_type": "tables"}, {"head": "таблицы", "head_type": "concept", "relation": "SHOWS_DATA", "tail": "данные по мероприятиям ГТМ", "tail_type": "data"}, {"head": "таблицы", "head_type": "concept", "relation": "GROUPED_BY", "tail": "различные группировки данных", "tail_type": "categories"}, {"head": "таблицы", "head_type": "concept", "relation": "CONTAINS_COLUMN", "tail": "столбец «Расчет»", "tail_type": "column"}, {"head": "факт/план", "head_type": "format", "relation": "CALCULATED_AS", "tail": "расчет

100%|██████████| 61/61 [08:32<00:00,  8.40s/it]


In [4]:
from langchain.chains import GraphCypherQAChain

qa_model = ChatOllama(
    model="llama3.1",
    callback_manager = callback_manager, 
    temperature=0.0, 
    # num_predict = 128
)

chain = GraphCypherQAChain.from_llm(
    qa_model, graph=graph, verbose=True, return_intermediate_steps = True,
)

C:\Users\mecco\AppData\Local\Temp\ipykernel_22812\2195666223.py:3: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  qa_model = ChatOllama(


In [5]:
graph.refresh_schema()

In [6]:
chain.invoke({"query": """Что такое бригада?
"""})



> Entering new GraphCypherQAChain chain...
MATCH (b:Brigada) RETURN b

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownLabelWarning} {category: UNRECOGNIZED} {title: The provided label is not in the database.} {description: One of the labels in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing label name is: Brigada)} {position: line: 1, column: 10, offset: 9} for query: 'MATCH (b:Brigada) RETURN b'


Generated Cypher:
MATCH (b:Brigada) RETURN b
Full Context:
[]
I don't know the answer.
> Finished chain.


{'query': 'Что такое бригада?\n',
 'result': "I don't know the answer.",
 'intermediate_steps': [{'query': 'MATCH (b:Brigada) RETURN b'},
  {'context': []}]}

In [25]:

driver = GraphDatabase.driver(
    uri = os.environ["NEO4J_URI"],
    auth = (os.environ["NEO4J_USERNAME"],
            os.environ["NEO4J_PASSWORD"]))
session = driver.session()

result_graph = session.run("MATCH (s)-[r:!MENTIONS]->(t) WHERE LOWER(s.id) =~ '.*скважина.*' RETURN s,r,t").graph()

widget = GraphWidget(graph = result_graph)
widget.node_label_mapping = 'id'
widget

GraphWidget(layout=Layout(height='750px', width='100%'))

In [7]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain


CYPHER_GENERATION_TEMPLATE = """Задача:Сгенерируй запрос Cypher к графовой базе данных.
Инструкции:
Используй в схеме только предоставленные типы отношений и свойства.
Не используй другие типы отношений или свойства, которые не предусмотрены.
Схема:
{schema}
Примечание: Не включай в свои ответы никаких объяснений или извинений.
Не отвечай на вопросы, в которых может потребоваться что-то еще, кроме построения утверждения на языке Cypher.
Не включай никакой текст, кроме сгенерированного утверждения Cypher.
Примеры: Вот несколько примеров сгенерированных утверждений Cypher для конкретных вопросов:
# Что такое нефть?
MATCH (s)-[r:!MENTIONS]->(t) WHERE LOWER(s.id) =~ '.*нефть.*' RETURN s,r,t

# Что такое газ?
MATCH (s)-[r:!MENTIONS]->(t) WHERE LOWER(s.id) =~ '.*газ.*' RETURN s,r,t

# Кто такой оператор?
MATCH (s)-[r:!MENTIONS]->(t) WHERE LOWER(s.id) =~ '.*оператор.*' RETURN s,r,t

Сам вопрос:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

cypher_llm = ChatOllama(
    model="llama3.1",
    callback_manager = callback_manager, 
    temperature=0.0, 
    # num_predict = 128
)

qa_llm = ChatOllama(
    model="rusmodel",
    callback_manager = callback_manager, 
    temperature=0.0, 
    # num_predict = 128
)

chain = GraphCypherQAChain.from_llm(
    cypher_llm = cypher_llm, 
    qa_llm = qa_llm,
    graph=graph, 
    verbose=True, 
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    validate_cypher=True,
)

C:\Users\mecco\AppData\Local\Temp\ipykernel_22812\1752102283.py:31: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  cypher_llm = ChatOllama(
C:\Users\mecco\AppData\Local\Temp\ipykernel_22812\1752102283.py:38: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  qa_llm = ChatOllama(


In [8]:
chain.run("Что такое мероприятия?")

C:\Users\mecco\AppData\Local\Temp\ipykernel_22812\2514515275.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  chain.run("Что такое мероприятия?")




> Entering new GraphCypherQAChain chain...
MATCH (s)-[r:!MENTIONS]->(t) WHERE LOWER(s.id) =~ '.*мероприятия.*' RETURN s,r,tGenerated Cypher:
MATCH (s)-[r:!MENTIONS]->(t) WHERE LOWER(s.id) =~ '.*мероприятия.*' RETURN s,r,t
Full Context:
[{'s': {'id': 'мероприятия'}, 'r': ({'id': 'мероприятия'}, 'REQUIRES_INPUT', {'id': 'контекстное меню добавить ремонт'}), 't': {'id': 'контекстное меню добавить ремонт'}}, {'s': {'id': 'мероприятия'}, 'r': ({'id': 'мероприятия'}, 'HAS_STEP', {'id': 'ГРП'}), 't': {'id': 'ГРП'}}, {'s': {'id': 'мероприятия'}, 'r': ({'id': 'мероприятия'}, 'HAS_PARALLEL_REPAIRS', {'id': 'скважина'}), 't': {'id': 'скважина'}}, {'s': {'id': 'мероприятия'}, 'r': ({'id': 'мероприятия'}, 'HAS_STEP', {'id': 'скважина'}), 't': {'id': 'скважина'}}, {'s': {'id': 'мероприятия'}, 'r': ({'id': 'мероприятия'}, 'REQUIRES_INPUT', {'id': 'панели изменения мероприятия'}), 't': {'id': 'панели изменения мероприятия'}}, {'s': {'id': 'мероприятия'}, 'r': ({'id': 'мероприятия'}, 'EDITED_BY', {'i

'Мероприятия - это события или действия, которые происходят в контексте Neo4j. Они могут включать в себя различные типы действий, такие как добавление ремонта, изменение шагов, добавление панели изменения мероприятия, добавление работы в мероприятие и т.д.'